In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras
import keras.layers as lay
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from src.generator import Generator
from src.discriminator import Discriminator
from src.aux_fn import train_step

In [ ]:
(trainx,trainy),(_,_) = keras.datasets.cifar10.load_data()
trainx = trainx[trainy[:,0]==1]
trainx = (tf.cast(trainx,tf.float32)-127.5)/127.5
data4 = tf.data.Dataset.from_tensor_slices((trainx)).map(lambda x: tf.image.resize(x,(4,4))).cache().shuffle(2**10).map(lambda x: tf.image.random_flip_left_right(x)).batch(2**8,drop_remainder=True).prefetch(tf.data.AUTOTUNE)
data8 = tf.data.Dataset.from_tensor_slices((trainx)).map(lambda x: tf.image.resize(x,(8,8))).cache().shuffle(2**10).map(lambda x: tf.image.random_flip_left_right(x)).batch(2**8,drop_remainder=True).prefetch(tf.data.AUTOTUNE)
data16 = tf.data.Dataset.from_tensor_slices((trainx)).map(lambda x: tf.image.resize(x,(16,16))).cache().shuffle(2**10).map(lambda x: tf.image.random_flip_left_right(x)).batch(2**8,drop_remainder=True).prefetch(tf.data.AUTOTUNE)
data32 = tf.data.Dataset.from_tensor_slices((trainx)).map(lambda x: tf.image.resize(x,(32,32))).cache().shuffle(2**10).map(lambda x: tf.image.random_flip_left_right(x)).batch(2**8,drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
gen = Generator(name='Generator')
dis = Discriminator(name='Discriminator')
gen.build()
dis.build()
# gen.summary()
# dis.summary()

In [ ]:
bce = keras.losses.BinaryCrossentropy()
opt = [keras.optimizers.Adam(1e-4,.5) for i in range(2)]

# Treinamento 4x4

In [ ]:
for i in range(120+1):
    for batch in data4:
        gl, dl = train_step([gen,dis],batch,bce,opt,stage=0)
    if i%10 == 0: print(f'EP = {i}; Gloss = {gl:.4f}; Dloss = {dl:.4f};')

In [ ]:
img = gen(tf.random.normal((1,128),0,1),stage=0)
img = np.uint8(img[0]*127.5+127.5)
plt.imshow(img)
plt.show()

# Treinamento 8x8

In [ ]:
for i in range(120+1):
    # gen.fade_in[0].alpha.assign(i/120.)
    # dis.fade_in[0].alpha.assign(i/120.)
    for batch in data8:
        gl, dl = train_step([gen,dis],batch,bce,opt,stage=1)
    if i%10 == 0: print(f'EP = {i}; Gloss = {gl:.4f}; Dloss = {dl:.4f};')

In [ ]:
img = gen(tf.random.normal((1,128),0,1),stage=1)
img = np.uint8(img[0]*127.5+127.5)
plt.imshow(img)
plt.show()

# Treinamento 16x16

In [ ]:
for i in range(120+1):
    # gen.fade_in[1].alpha.assign(i/120.)
    # dis.fade_in[1].alpha.assign(i/120.)
    for batch in data16:
        gl, dl = train_step([gen,dis],batch,bce,opt,stage=2)
    if i%10 == 0: print(f'EP = {i}; Gloss = {gl:.4f}; Dloss = {dl:.4f};')

In [ ]:
img = gen(tf.random.normal((1,128),0,1),stage=2)
img = np.uint8(img[0]*127.5+127.5)
plt.imshow(img)
plt.show()

# Treinamento 32x32

In [ ]:
for i in range(120+1):
    # gen.fade_in[2].alpha.assign(i/120.)
    # dis.fade_in[2].alpha.assign(i/120.)
    for batch in data32:
        gl, dl = train_step([gen,dis],batch,bce,opt,stage=3)
    if i%10 == 0: print(f'EP = {i}; Gloss = {gl:.4f}; Dloss = {dl:.4f};')

In [ ]:
f = 10

imgs = gen(tf.random.normal((f**2,128),0,1),stage=3)
imgs = np.uint8(imgs*127.5+127.5)
fig, ax = plt.subplots(f,f,figsize=(8,8))
ax = ax.ravel()
for i in range(f*f):
    ax[i].imshow(imgs[i])
    ax[i].set_axis_off()
plt.tight_layout(pad=0)
plt.show()

In [ ]:
img = gen(tf.random.normal((1,128)),stage=3)
plt.imshow(np.uint8(img[0]*127.5+127.5))
plt.show()